In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from ydata_profiling import ProfileReport


In [2]:
DATA_PATH = '../data/train.csv'
df = pd.read_csv(DATA_PATH)
df.head()

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,999,D-penicillamine,21532,M,N,N,N,N,2.3,316.0,3.35,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,D
1,1,2574,Placebo,19237,F,N,N,N,N,0.9,364.0,3.54,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,C
2,2,3428,Placebo,13727,F,N,Y,Y,Y,3.3,299.0,3.55,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,D
3,3,2576,Placebo,18460,F,N,N,N,N,0.6,256.0,3.50,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,C
4,4,788,Placebo,16658,F,N,Y,N,N,1.1,346.0,3.65,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,C


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7905 entries, 0 to 7904
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             7905 non-null   int64  
 1   N_Days         7905 non-null   int64  
 2   Drug           7905 non-null   object 
 3   Age            7905 non-null   int64  
 4   Sex            7905 non-null   object 
 5   Ascites        7905 non-null   object 
 6   Hepatomegaly   7905 non-null   object 
 7   Spiders        7905 non-null   object 
 8   Edema          7905 non-null   object 
 9   Bilirubin      7905 non-null   float64
 10  Cholesterol    7905 non-null   float64
 11  Albumin        7905 non-null   float64
 12  Copper         7905 non-null   float64
 13  Alk_Phos       7905 non-null   float64
 14  SGOT           7905 non-null   float64
 15  Tryglicerides  7905 non-null   float64
 16  Platelets      7905 non-null   float64
 17  Prothrombin    7905 non-null   float64
 18  Stage   

#### La tabla anterior muestra que 7905 filas y 20 columnas, de las cuales no hay valores nulos.

#### Vamos a rectificar los datos nulos

In [5]:
for i in df.columns:
    print(i, df[i].unique()) 

id [   0    1    2 ... 7902 7903 7904]
N_Days [ 999 2574 3428 2576  788  703 1300 1615 2050 2615 3581 1614 1847 1153
  904 1212 1967 1592 1481 3358 3092  326 2363 1152 3577  799 1832 4467
 2301  943 1882 3244 3090  850 2168 1786 1455  334 1363 3170 4032 2835
 1434 3913  994 1666 3820 1149  930 2157 2555 1776 4050   51 3445 1765
 3458  552 2769 4453 2504  980  853  786 3395 2357  549 1447  732  597
  708 2105 1295 2419 3098  769 1083 1785 1657 1418  737 1701 2598 2891
 1234 3933 1271 3099 1831 3707  191 2318 1321 1420  794 1690 3672  837
 1349 2556 4523 1170 1191 1367 1412 4509 2294 2976 1303 1250 2255 1293
 3149 1433 2583 3086 2297 1908 1000 2224 4190 4500 1978 2386 2624 4079
 1568 3422 4365 3336 1576 2990  762 1951 3839  974 2403 1649 2022 1770
 3297 1569 1297 2713 1282   77 1197 4039 1302  304 1487 2365 2170  321
 1725  971 1077 3468 2350 1677 1216  385 2657 1945 1408 1444 1702 1030
  179 1360 1525 1682  877  890 1165 2330 1783  939  216 4556  735 2449
 1932 3574 3853  223 3282 1012 